<a href="https://colab.research.google.com/github/JRiveraB23/SyS05/blob/main/SerieyTFAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

- Se debe incluir el link del video a procesar:

In [ ]:
link="https://www.youtube.com/watch?v=ctb9uuxJVHQ"
!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}

- Se convierte el audio a .wav para procesar:

In [ ]:
!ffmpeg -y -i audio.mp3 output.wav

Librería para manipulación de archivos de audio

In [ ]:
!pip install soundfile

In [ ]:
import soundfile as sf # para instalar pip install soundfile
#lee archivos wav
nombre_out = "output.wav"
x, fs = sf.read(nombre_out)
# read speech signal from file
print('Frecuencia de muestreo %.2f[Hz]\naudio %s' % (fs,nombre_out))

El audio viene codificado en dos canales:

In [ ]:
x.shape

In [ ]:
from IPython.display import Audio
ns = 30 #se reproducen los primeros ns segundos
Audio(x[:int(fs*ns),:].T,rate=fs)

- Se gráfica un intervalo de la señal en el tiempo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xpro = x.copy() #copiar archivos para procesar
ti = 10 #tiempo incio a procesar seg
tf = 15 #tiempo final a procesar seg
xs = xpro[int(ti*fs):int((tf*fs)),:]

tt = np.arange(ti,tf,1/fs) # vector de tiempo
plt.plot(tt,xs)
plt.legend(('canal 1','canal 2'))
plt.xlabel('$t[s]$')
plt.ylabel('$x(t)$')
plt.legend()
plt.show()

- Se cálcula el espectro de Fourier del segmento de audio escogido sobre cada canal (sonido estereo).

In [ ]:
Xw = np.fft.rfft(xs,axis=0) # axis=0 permite aplicar fft por cada columna de xpro
#Xwc1 = np.fft.rfft(xs[:,0])
#Xwc2 = np.fft.rfft(xs[:,1])
vf = np.fft.rfftfreq(np.size(xs,0),1/fs) #se crea el vector de frecuencias
plt.plot(vf,abs(Xw))#se grafica la magnitud
plt.legend(('canal 1','canal 2'))
plt.title(r'Espectro audio original')
plt.xlabel(r'$f[Hz]$',fontsize = 14)
plt.ylabel(r'$|X[n]|$',fontsize = 14)
plt.show()

- En este caso, se plantea un filtro pasa banda, apagando los armónicos que no nos interesa.

In [ ]:
#filtrar espectro
Xwf = Xw.copy()
f1 = 500 #frecuencia en Hz corte 1
f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf > f1) & (vf < f2)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwf[ind,:] = 0
plt.plot(vf,abs(Xwf))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro pasa bajos

In [ ]:
#filtrar espectro
Xwfb = Xw.copy()
f1 = 5000 #frecuencia en Hz corte 1
#f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf < f1)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfb[ind,:] = 0
plt.plot(vf,abs(Xwfb))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro pasa altas

In [ ]:
#filtrar espectro
Xwfa = Xw.copy()
f1 = 5000 #frecuencia en Hz corte 1
#f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf > f1)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfa[ind,:] = 0
plt.plot(vf,abs(Xwfa))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro rechaza banda

In [ ]:
#filtrar espectro
Xwfrr = Xw.copy()
f1 = 100 #frecuencia en Hz corte 1
f2 = 6000#frecuencia en Hz corte 2
ind = ((vf > f1) & (vf < f2)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfrr[ind,:] = 0
plt.plot(vf,abs(Xwfrr))
plt.legend(('canal 1','canal 2'))
plt.show()

- Ahora, se reconstruye la señal mediante la fft inversa.

In [ ]:
xe2 = np.fft.irfft(Xwf,axis=0) #fft inversa sobre los dos canales de audio
xe2b = np.fft.irfft(Xwfb,axis=0) #fft inversa sobre los dos canales de audio
xe2a = np.fft.irfft(Xwfa,axis=0) #fft inversa sobre los dos canales de audio
xe2r = np.fft.irfft(Xwfrr,axis=0) #fft inversa sobre los dos canales de audio

In [ ]:
Audio(xe2[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa banda

In [ ]:
Audio(xe2b[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa baja

In [ ]:
Audio(xe2a[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa alta

In [ ]:
Audio(xe2r[:int(fs*ns),:].T,rate=fs)#señal del filtro rechaza banda